In [ ]:
import os
 
######################################################################## Class creation ##########################################################################
class Menu:
    '''Creates a menu which displays the first letter of each method passed to the object then calls 
        the corresponding method when that is selected by the user.'''
    
    def __init__(self, option_1, option_2, option_3, option_4, method_1, method_2, method_3, method_4):
        '''Allows for the creation of main menus and sub-menus'''
       
        self.option_1 = option_1
        self.option_2 = option_2
        self.option_3 = option_3
        self.option_4 = option_4
       
        
        self.method_1 = method_1
        self.method_2 = method_2
        self.method_3 = method_3  
        self.method_4 = method_4 
 
    def show_menu(self): # Algorithm for show_menu method inspired by code in HyperionDev (2018). 
        '''Displays the menu to the users, with options numbers corresponding
           to matching method numbers.''' 
        
        while True:                       
        # presenting the menu to the user and
        # making sure that the user input is converted to lower case.
          print()
          menu = input(f'''Select one of the following Options below:         
                {self.option_1[0].lower()} - {self.option_1}
                {self.option_2[0].lower()} - {self.option_2}
                {self.option_3[0].lower()} - {self.option_3}          
                {self.option_4[0].lower()} - {self.option_4}        
        :       ''').lower()             
 
          if menu == self.option_1[0].lower():
            self.method_1()               
 
          elif menu == self.option_2[0].lower():
            self.method_2()                                 
              
          elif menu == self.option_3[0].lower():
            self.method_3()
                                                 
          elif menu == self.option_4[0].lower():
            self.method_4()               
                
          else:
            print("\n!! Choose a valid option. !!\n")           
  
class Phonebook:           
    '''Creates a phonebook in which contacts can be added, edited, viewed,
       or searched for.'''
    def __init__(self, txt_file, main_menu, sub_menu_1, sub_menu_2):
       
        # Text file to save to
        self.txt_file = txt_file
       
        # Menus to display to navigate methods.
        self.main_menu = main_menu
        self.sub_menu_1 = sub_menu_1
        self.sub_menu_2 = sub_menu_2
       
        # List of dictionaries in which to save contacts.      
        self.contact_list = []
       
        # Instance variables needed to access current contact searched for.
        self.current_contact = ""               
        self.new_name = ""
        self.new_number = 0
        self.current_index = 0
        self.search_name = ""
 
        # Flags.
        self.search_success = False
        self.adding_new_contact = False
        self.name_checked = False
        self.details_checked = False
        self.editing_name_only = False       
        self.editing_number_only = False
        self.editing_both = False
        self.discrete_searching = False
        self.search_already_done = False
       
        # Create the .txt file if it doesn't exist
        if not os.path.exists(txt_file):
            with open(txt_file, "w") as default_file:
                pass
       
        # Make sure self.contact_list contains latest contact details that are
        # saved in the .txt file.
        self.update_contact_list(txt_file)                  
 
 
    def view_contacts(self):
        '''Displays the contacts in alphabetical order.'''       
        
        # Reset flags if arriving here after a search.
        if self.search_already_done:
            self.reset_flags_and_variables()
       
        self.update_contact_list(self.txt_file)
               
        sorted_contact_list = self.bubble_sort(self.contact_list) # Sort contact list alphabetically by name.               
 
        if sorted_contact_list == []:
            print("\n!! Your contacts list is empty !!\n")
        else:           
            for contact in sorted_contact_list:                                       
                print(f"\nname: {contact['name']}\n")
                print(f"number: {contact['number']}\n\n")
                print("------------------------\n")                                 
            
            self.sub_menu_1()
   
 
    def bubble_sort(self, contact_list):
        '''Sorts the contact list alphabetically by name.'''
        swapped = False
 
        n = len(contact_list)
 
        for i in range(n):
 
            # Nested loop to iterate through each dictionary, indexed as 'j'
            for j in range(0, n-i-1):
 
                # Variables for name values in dictionaries next to each other in contact_list.
                name1 = contact_list[j]["name"]
                name2 = contact_list[j+1]["name"]
 
                # Swap contact order if name value closer to 'a' in alphabet.
                if name1 > name2:
 
                    contact_list[j], contact_list[j+1] = contact_list[j+1], contact_list[j]
                    swapped = True
 
            # Exit outer loop and stop comparing if single swap not made (i.e. list already in order).
            if not swapped:
                break
 
        return contact_list   
    
    
    def binary_search(self, name, contact_list, start=0, end=None): # OpenAI ChatGPT (2024)
        '''searches for inputted name by dividing contact_list in half each time.'''
 
        if end is None:
            end = len(contact_list) - 1
 
        if start <= end:           
            halfway = (start + end) // 2 # Use integer division to avoid error with index due to float.             
            
            # Ensure rest of program - e.g. delete() and embedded_delete() - can access the contact found within self.current_contact and self.current_index variable.
            if contact_list[halfway]["name"].lower() == name.lower():
                self.current_contact = contact_list[halfway]
                self.current_index = self.contact_list.index(self.current_contact)
                self.search_success = True
 
            elif contact_list[halfway]["name"].lower() > name.lower():
               
                return self.binary_search(name, contact_list, start, halfway - 1) # Search in the left half of the contact_list. 
            else:
               
                return self.binary_search(name, contact_list, halfway + 1, end) # Search in the right half of the contact_list.
        else:
           
            return # Return if not found, leaving self.search_success as False
   
 
    def search(self):
        '''Searches for a contact (by name) when 'search' is discretely selected
           and then displays name and number.'''                                                     
        
        # If directed here from 'search again' option, flags need to be reset.
        if self.search_already_done:
            self.reset_flags_and_variables()
 
        self.discrete_searching = True               
        
        self.update_contact_list(self.txt_file) 
        
        if self.contact_list == []:
            print("\n!! Your contacts list is empty !!\n")
            self.main_menu()
           
        else:                       
            self.binary_search(input("\nEnter contact name to search for: "), self.bubble_sort(self.contact_list))                                              
                                                                                                                                                  
        if self.search_success == False:
            print(f"\n!! This contact has not yet been added !!\n")
                           
            self.main_menu() # Return to main menu to allow user to search again.                       
            
        else:
            self.display_success_of_search_or_change_or_addition()
                           
            self.search_success = False # Reset flag.
           
            # Keep track of when a search has already been done so that different logic is applied
            # after calling methods from next sub-menu e.g. delete *this* contact.
            self.search_already_done = True
                                                      
            self.sub_menu_2()             
    
    def delete_contact(self):
        '''Deletes contact details for given name.'''
       
        # Reset flags if arriving here after a search.
        if self.search_already_done:
            self.reset_flags_and_variables()
       
        name = input("\nType the name of the contact that you would like to delete: ")                 
 
        self.binary_search(name, self.bubble_sort(self.contact_list)) # Search to find out if name exists and assign the dictionary to self.current_contact if it does.
       
        if self.search_success == False:
           
            print("\n!! This contact has not yet been added !!\n")
           
            self.main_menu           
 
        else:
           
            self.contact_list.remove(self.current_contact)                                
        
            print("\n\n** Contact deleted **\n\n")       
                                    
            self.write_to_txt_file(self.txt_file)
            
        self.reset_flags_and_variables()
       
        self.main_menu
 
    def insert_contact(self, name, number):
        '''Inserts an edited contact or new contact into the self.contact_list list of dictionaries.'''
              
        if self.adding_new_contact:                       
            new_contact = {"name": name, "number": number}               
            self.contact_list.append(new_contact)
        else:
            self.contact_list[self.current_index]["name"] = name
            self.contact_list[self.current_index]["number"] = number
           
        return
   
 
    def add_contact(self):
        '''Adds new contact to the self.contact_list instance variable
           or saves edited contacts to said list.'''                               
 
        # Find out if user is adding a new contact (as opposed to editing) and perform checks for valid data
        # input if so.        
        if self.editing_name_only == False and self.editing_number_only == False and self.editing_both == False and self.details_checked == False:
           
            self.adding_new_contact = True                       
                        
            self.input_a_valid_name() # This check will prompt for a name, check its validity then lead onto prompting for a number before checking number validity.     
               
        elif self.adding_new_contact:
                                                          
            self.insert_contact(self.new_name, self.new_number) # Add new_name and new_number as a dictionary if adding a new contact after name and number validity 
                                                                # check (i.e. if not in editing mode).                                                                      
            
        self.display_success_of_search_or_change_or_addition()
       
        self.write_to_txt_file(self.txt_file)               
                
        self.reset_flags_and_variables()      
                
        self.main_menu()
   
 
    def input_a_valid_name(self):
        '''Checks that the name inputted is valid (used in add_contact,
           edit_name and edit_number).'''
   
        while True:                           
            
            # Let user know they are entering a 'new' name if editing or adding new contact.
            if self.adding_new_contact or self.editing_name_only or self.editing_both:
                self.new_name = (input("\nEnter new contact name: "))
                
            else:
                self.new_name = (input("\nEnter contact name: "))
                
            # Check ASCII code of characters and add to this list if character is not a letter or a space.
            non_letter = [char for char in self.new_name.lower() if ord(char) < 97 or ord(char) > 122] 
    
            non_letter_or_non_space = [char for char in non_letter if ord(char) != 32] # Check ASCII code of non-letters to see if any are spaces and add here.           
    
            num_of_spaces = [char for char in self.new_name if ord(char) == 32] # Check ASCII code of characters and add to this list if character is a space.            
    
            if self.new_name.isnumeric() or self.name_is_a_float(self.new_name):                               
                print("\n!! You need to enter a name, not a number !!\n")
            
            elif len(non_letter_or_non_space) > 0:                
                print("\n!! You need to enter a name using letters !!\n")                            
    
            elif self.new_name.lower() == "back":
                print("\n!! This is a menu option so can't be used as a name - that's just confusing !!\n")               
                
            elif len(self.new_name) > 20:
                print("\n!! You have entered too many characters. Try again !!")
                           
            # Print error message if all characters entered are spaces.
            elif len(num_of_spaces) == len(self.new_name):
                print("\n!! You did not enter a name. Try again !!")
                
            else:               
            
                self.binary_search(self.new_name, self.bubble_sort(self.contact_list)) # Search to check if contact already exists.
                       
                if self.search_success:
                    print("\n!! This contact already exists. Pick a different name !!\n")           
                                
                    self.search_success = False # Reset search_success flag so as not to disrupt logic further into program.               
                
                else:                   
                    self.name_checked = True
                    
                    if self.adding_new_contact:
                        self.input_a_valid_number()
                   
                    return               
           
                     
                    
    def name_is_a_float(self, name):
        '''Checks if the inputted name is a float, as part of
           check_name_is_valid() method.'''
        try:
            float(name)
            return True
       
        except ValueError:
            return False
 
 
    def input_a_valid_number(self):
        '''Checks that the number inputted is valid (used in
           add_contact, edit_name and edit_number).'''               
     
        while True:                       
        
            self.new_number = str(input("Enter new contact number: ")).lower() # Change number entered to a string so that '0' is not ignored as first digit.
            
            non_integer = [char for char in self.new_number if ord(char) < 48 or ord(char) > 57] # Check ASCII code of characters and add to this list if they are not a number.
        
            digit_list = [x for x in self.new_number] # Change number inputted to list of digits so their length can be counted.
        
            # Do not progress if non-integer characters were found.
            if len(non_integer) != 0:
                print("\n!! You need to enter a number !!\n")                
                                           
            elif len(digit_list) > 11:
             print("\n!! You have entered too many digits. Try again !!")
                                               
            else:
                self.details_checked = True
               
                if self.adding_new_contact:
                    self.add_contact()
                else:
                    return
 
    def edit_choices(self):
        '''Directs user to correct edit method.''' # Algorithm for edit_choices method inspired by code in HyperionDev (2018). 
       
        while True:
            # Prompt user to input name to edit if they haven't selected 'edit this contact' from
            # the search sub-menu.
            if self.search_already_done == False:
               
                self.search_name = input('''\nType the name of the contact that you would like to edit or type 'back' to go back: ''')           
                        
                self.binary_search(self.search_name, self.bubble_sort(self.contact_list)) # Find inputted name from self.contact_list and save contact to self.current_contact.
               
            if self.search_already_done == False and self.search_success == False:
                print(f"\n!! This contact has not yet been added !!\n")
                self.sub_menu_1()
           
            elif self.search_name.lower() == "back":
                               
                self.search_success = False               
                main_menu()
       
            else:
                option = input("Type 'na' to edit 'name', 'nu' to edit 'number', or 'b' to edit both: ").lower()
               
                self.search_success = False
               
                if option == "b":
                    self.editing_both = True                   
                    self.edit_name() # edit_name() logic will lead user to edit_number() if 'both' selected.
                elif option == "na":
                    self.editing_name_only = True    
                    self.edit_name()
                elif option == "nu":
                    self.editing_number_only = True
                    self.edit_number()
                else:
                    print("!! Choose a valid option !!")
                           
                
    def edit_name(self):
        '''Allows the user to edit a name.'''                                          
                       
        self.input_a_valid_name()              
                        
        if self.editing_name_only:
            self.insert_contact(self.new_name, self.contact_list[self.current_index]["number"])
            self.add_contact() # Changes are saved here.
        else:
            self.edit_number() # Move to edit number if 'edit both' or 'add contact' option chosen.
 
 
    def edit_number(self):       
        '''Allows the user to edit a number.'''       
                                
        self.input_a_valid_number()               
        
        if self.editing_number_only:
            self.insert_contact(self.contact_list[self.current_index]["name"], self.new_number)
       
        else:
            self.insert_contact(self.new_name, self.new_number)
           
        self.add_contact() # Details are now saved within add_contact                       
        
                               
    def delete_current_contact(self):
        '''Deletes the current contact when selected from the discrete search sub-menu.'''
       
        self.contact_list.remove(self.current_contact)
                       
        print("\n\n** Contact deleted **\n\n")
       
        self.write_to_txt_file(self.txt_file)               
        
        self.reset_flags_and_variables()
       
        self.main_menu()
       
 
    def reset_flags_and_variables(self):
        '''Resests flags and variables after contacts added'''
   
        # Reset flags.
        self.details_checked = False
        self.name_checked = False
        self.adding_new_contact = False               
        self.search_success = False
        self.editing_name_only = False       
        self.editing_number_only = False
        self.editing_both = False
        self.discrete_searching = False
        self.search_already_done = False
 
        # Reset instance variables.
        self.new_number = 0
        self.current_index = 0
        self.current_contact = ""               
        self.new_name = ""       
        self.search_name = ""      
 
 
    def display_success_of_search_or_change_or_addition(self):
        '''Displays a confirmation of searched-for, added or edited contact.'''
       
        if self.discrete_searching and self.search_already_done == False:
            print("\n** Contact found **\n")
            print(f"name: {self.contact_list[self.current_index]['name']}\n")
            print(f"number: {self.contact_list[self.current_index]['number']}\n")           
        elif self.editing_name_only:
            print("\n** Contact saved **\n")
            print(f"name: {self.new_name}\n")
            print(f"number: {self.contact_list[self.current_index]['number']}\n")               
        elif self.editing_number_only:
            print("\n** Contact saved **\n")
            print(f"name: {self.contact_list[self.current_index]['name']}\n")
            print(f"number: {self.new_number}\n")
        else:           
            print("\n** Contact saved **\n")
            print(f"name: {self.new_name}\n")
            print(f"number: {self.new_number}\n")                                                 
    
        
    def write_to_txt_file(self, txt_file):
        '''Writes changes made to the contacts in self.contact_list to a .txt file.''' # Algorithm for write_to_txt_file method inspired by code found in HyperionDev (2018).
       
        # Change each dictionary in self.contact_list to a sublist of string pairs so that it can be written to .text file.
        with open(txt_file, "w") as contacts_file:
            contact_list_to_write = []
            for c in self.contact_list:
                str_attributes = [
                    c['name'],
                    str(c['number'])                   
                ]
                contact_list_to_write.append(";".join(str_attributes))
            contacts_file.write("\n".join(contact_list_to_write))
 
    def update_contact_list(self, txt_file):
        '''Saves the updated .txt file to contact_list.''' # Algorithm for update_contact_list method inspired by code found in HyperionDev (2018).
       
        with open(txt_file, "r+") as current_contacts:
                       
            self.contact_data = current_contacts.read().split("\n") # Read the contents and move them into a list, separated where there is a newline character.
            self.contact_data = [c for c in self.contact_data if c != ""] # Reformat list so that it doesn't contain blank spaces.
           
            self.contact_list = [] # Clear self.contact_list of old data.
           
            for contact_str in self.contact_data:
                curr_contacts = {}
                               
                contact_components = contact_str.split(";")                
                curr_contacts['name'] = contact_components[0]
                curr_contacts['number'] = contact_components[1]               
                                                                  
                self.contact_list.append(curr_contacts)                   
 
 
############################################# Instantiating objects ###########################################################################################
 
my_phonebook = Phonebook("contacts.txt", None, None, None) # Instantiate the phonebook class with placeholders for the menus that are yet to be instantiated.               
 
# Instantiate the main menu and sub-menus.
main_menu = Menu("Add a contact", "View contacts", "Search for a contact", "Exit", my_phonebook.add_contact, my_phonebook.view_contacts,
                 my_phonebook.search, exit)
 
view_menu = Menu("Add a contact", "Edit a contact", "Delete a contact", "Home", my_phonebook.add_contact, my_phonebook.edit_choices,
                my_phonebook.delete_contact, main_menu.show_menu)
 
search_menu = Menu("Delete this contact", "Edit this contact", "Search again", "Home", my_phonebook.delete_current_contact, my_phonebook.edit_choices,
                my_phonebook.search, main_menu.show_menu)
 
 
# Update the menus in my_phonebook object now that they have been instantiated.
my_phonebook.main_menu = main_menu.show_menu
my_phonebook.sub_menu_1 = view_menu.show_menu
my_phonebook.sub_menu_2 = search_menu.show_menu
 
########################################### Initialising the program #########################################################################################
 
main_menu.show_menu() # Display the main menu as the first page of the program.













Select one of the following Options below:         
                a - Add a contact
                v - View contacts
                s - Search for a contact          
                e - Exit        
        :        a

Enter new contact name:  Zero
Enter new contact number:  07414444555



** Contact saved **

name: Zero

number: 07414444555


